In [2]:
import cv2
import numpy as np
from keras.models import load_model
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk

# Define relative paths for the models
dog_vs_cat_model_path = './dog_vs_cat_100_epochs.model'
expression_model_path = './animal_expression_100_epochs.model'

# Load dog vs. cat classification model
dog_vs_cat_model = load_model(dog_vs_cat_model_path)

# Load animal expression detection model
expression_model = load_model(expression_model_path)

# Define labels for expressions
expression_labels = ['happy', 'Sad', 'Angry', 'Other']

def integrate_models(frame):
    # Resize frame to match input size of the models
    resized_frame = cv2.resize(frame, (48, 48))

    # Preprocess frame for dog vs. cat model
    input_frame_dog_vs_cat = np.expand_dims(resized_frame, axis=0) / 255.0

    # Predict dog vs. cat
    dog_vs_cat_prediction = dog_vs_cat_model.predict(input_frame_dog_vs_cat)
    is_dog = dog_vs_cat_prediction[0][0] > 0.5
    is_cat = not is_dog

    if is_dog or is_cat:
        # Preprocess frame for expression detection model
        input_frame_expression = np.expand_dims(resized_frame, axis=0)

        # Predict expression
        expression_prediction = expression_model.predict(input_frame_expression)
        predicted_expression = expression_labels[np.argmax(expression_prediction)]

        # Generate final output
        if is_dog:
            animal_label = "Dog"
        else:
            animal_label = "Cat"
        
        final_output = f"{animal_label}: {predicted_expression}"
    else:
        final_output = "Neither dog nor cat detected"
    
    return final_output

class Application(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Animal Classifier")
        self.geometry("800x600")

        self.label = tk.Label(self, text="Select an image or video to classify:")
        self.label.pack(pady=10)

        self.btn_image = tk.Button(self, text="Select Image", command=self.open_image)
        self.btn_image.pack(pady=5)

        self.btn_video = tk.Button(self, text="Select Video", command=self.open_video)
        self.btn_video.pack(pady=5)

        self.result_label = tk.Label(self, text="")
        self.result_label.pack(pady=10)

    def open_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png")])
        if file_path:
            self.process_image(file_path)

    def open_video(self):
        file_path = filedialog.askopenfilename(filetypes=[("Video files", "*.mp4")])
        if file_path:
            self.process_video(file_path)

    def process_image(self, image_path):
        # Read image
        image = cv2.imread(image_path)
        # Process image
        processed_output = integrate_models(image)
        self.result_label.config(text=processed_output)

    def process_video(self, video_path):
        cap = cv2.VideoCapture(video_path)
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            processed_output = integrate_models(frame)
            self.result_label.config(text=processed_output)
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        cap.release()

if __name__ == "__main__":
    app = Application()
    app.mainloop()
